In [ ]:
!pip install google-generativeai google-cloud-secret-manager

In [ ]:
from IPython import get_ipython
from IPython.display import Markdown, display

In [ ]:
! gcloud services enable secretmanager.googleapis.com

In [ ]:
import subprocess
import json
import pandas as pd
def get_projects_dataframe():
    # Command to get projects in JSON format
    command = ["gcloud", "projects", "list", "--format=json"]

    # Run the command and capture the output
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check for errors
    if result.returncode != 0:
        print("An error occurred while running gcloud projects list:")
        print(result.stderr)
        exit(1)

    # Parse the JSON output
    projects_data = json.loads(result.stdout)

    # Convert to pandas DataFrame
    df = pd.DataFrame(projects_data)

    return df

# Get the DataFrame
df = get_projects_dataframe()

# Display the DataFrame
projectNumber = df['projectNumber']
print(projectNumber)

In [ ]:
! gcloud secrets create gemini_api \
    --replication-policy="automatic"

**Now fetch your API key for Gemini from [this website](https://aistudio.google.com/app/apikey), click `Get API Key`.** and add it to your secret manager secret.

In [ ]:
from google.cloud import secretmanager

def access_secret_version(resource_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": resource_name})
    return response.payload.data.decode("UTF-8")


# Your secret's resource name
resource_name = f"projects/{projectNumber[0]}/secrets/gemini_api/versions/1"

# Access the secret
apiKey = access_secret_version(resource_name)

In [ ]:
import google.generativeai as genai

genai.configure(api_key=apiKey)

model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")
# print(response.text)
welcome_prompt = "You are a excellent developer in life science and healthcare research. The mission is to advise researchers with limited coding experience. Please format your response in markdown by default."

In [ ]:
def explain(cell_number):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        content = "Based on the input. Please concisely comment this code to explain each line. Ignore the run cell magic and just focus on the code or error " + ipython.user_ns['In'][cell_number] + "Do not add additional code"
        response = model.generate_content(welcome_prompt + content)

        return display(Markdown(response.text))
        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."
    

    
def modify(cell_number,modification):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        prompt = "Please modify the code "  + ipython.user_ns['In'][cell_number] + " to accomplish " + modification + "Ignore the run cell magic and just focus on the code. Assume all library has been loaded. Return only code."
        response = model.generate_content(prompt)

        return create_new_cell("%%R\n\n" + response.text)

        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."

def propose(suggest):
    response = model.generate_content("Please suggest code to accomplish " + suggest + ". Return only code.")
    #print(response.choices[0].message.content)
    #return display(Markdown(response.choices[0].message.content))
    return create_new_cell("\n\n" + response.text)
    
def create_new_cell(contents):
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)


In [ ]:
propose('write me code for openai')

In [ ]:
propose("I need to find the 25 most signficantly upregulated genes from a dataframe called gene. Here are the column names: ENTREZID, SYMBOL, GENENAME, logFC AveExpr, t, P.Value, adj.P.Val")

In [ ]:
propose("help me develop a nextflow workflow")